<a href="https://colab.research.google.com/github/ComponentSoftTeam/Mistral-Kubectl-Instruct/blob/main/notebooks/k8s_semantic_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install cohere tiktoken sentence_transformers transformers openai tqdm numpy torch pinecone-client datasets PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 99.9 MB/s 

In [2]:
import openai
from datetime import datetime
import hashlib
import re
import os
from tqdm import tqdm
import numpy as np

import logging

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)


## Set up Pinecone

In [3]:
from google.colab import userdata


pinecone_key = userdata.get('PINECONE_API_KEY')

INDEX_NAME = 'k8s-semantic-search'
NAMESPACE = 'default'

In [4]:
import pinecone

pinecone.init(api_key=pinecone_key, environment="gcp-starter")

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): controller.us-west1-gcp.pinecone.io:443
DEBUG:urllib3.connectionpool:https://controller.us-west1-gcp.pinecone.io:443 "GET /actions/whoami HTTP/1.1" 401 114
/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [5]:
if not INDEX_NAME in pinecone.list_indexes():
    pinecone.create_index(
        INDEX_NAME,  # The name of the index
        dimension=768,  # The dimensionality of the vectors
        metric='cosine',  # The similarity metric to use when searching the index
        pod_type='starter'  # The type of Pinecone pod
    )

# Store the index as a variable
index = pinecone.Index(INDEX_NAME)

In [6]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/multi-qa-mpnet-base-cos-v1')

DEBUG:jaxlib.mlir._mlir_libs:Initializing MLIR with module: _site_initialize_0
DEBUG:jaxlib.mlir._mlir_libs:Registering dialects from initializer <module 'jaxlib.mlir._mlir_libs._site_initialize_0' from '/usr/local/lib/python3.10/dist-packages/jaxlib/mlir/_mlir_libs/_site_initialize_0.so'>
DEBUG:jax._src.path:etils.epath found. Using etils.epath for file I/O.
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/multi-qa-mpnet-base-cos-v1
DEBUG:filelock:Attempting to acquire lock 138826682262832 on /root/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-mpnet-base-cos-v1/.gitattributes.lock
DEBUG:filelock:Lock 138826682262832 acquired on /root/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-mpnet-base-cos-v1/.gitattributes.lock


DEBUG:filelock:Attempting to release lock 138826682262832 on /root/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-mpnet-base-cos-v1/.gitattributes.lock
DEBUG:filelock:Lock 138826682262832 released on /root/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-mpnet-base-cos-v1/.gitattributes.lock
DEBUG:filelock:Attempting to acquire lock 138826693398816 on /root/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-mpnet-base-cos-v1/1_Pooling/config.json.lock
DEBUG:filelock:Lock 138826693398816 acquired on /root/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-mpnet-base-cos-v1/1_Pooling/config.json.lock


DEBUG:filelock:Attempting to release lock 138826693398816 on /root/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-mpnet-base-cos-v1/1_Pooling/config.json.lock
DEBUG:filelock:Lock 138826693398816 released on /root/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-mpnet-base-cos-v1/1_Pooling/config.json.lock
DEBUG:filelock:Attempting to acquire lock 138826682262832 on /root/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-mpnet-base-cos-v1/README.md.lock
DEBUG:filelock:Lock 138826682262832 acquired on /root/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-mpnet-base-cos-v1/README.md.lock


DEBUG:filelock:Attempting to release lock 138826682262832 on /root/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-mpnet-base-cos-v1/README.md.lock
DEBUG:filelock:Lock 138826682262832 released on /root/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-mpnet-base-cos-v1/README.md.lock
DEBUG:filelock:Attempting to acquire lock 138826682260000 on /root/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-mpnet-base-cos-v1/config.json.lock
DEBUG:filelock:Lock 138826682260000 acquired on /root/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-mpnet-base-cos-v1/config.json.lock


DEBUG:filelock:Attempting to release lock 138826682260000 on /root/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-mpnet-base-cos-v1/config.json.lock
DEBUG:filelock:Lock 138826682260000 released on /root/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-mpnet-base-cos-v1/config.json.lock
DEBUG:filelock:Attempting to acquire lock 138826682267680 on /root/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-mpnet-base-cos-v1/config_sentence_transformers.json.lock
DEBUG:filelock:Lock 138826682267680 acquired on /root/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-mpnet-base-cos-v1/config_sentence_transformers.json.lock


...

DEBUG:filelock:Attempting to release lock 138826682262592 on /root/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-mpnet-base-cos-v1/tokenizer.json.lock
DEBUG:filelock:Lock 138826682262592 released on /root/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-mpnet-base-cos-v1/tokenizer.json.lock
DEBUG:filelock:Attempting to acquire lock 138826682553952 on /root/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-mpnet-base-cos-v1/tokenizer_config.json.lock
DEBUG:filelock:Lock 138826682553952 acquired on /root/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-mpnet-base-cos-v1/tokenizer_config.json.lock


DEBUG:filelock:Attempting to release lock 138826682553952 on /root/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-mpnet-base-cos-v1/tokenizer_config.json.lock
DEBUG:filelock:Lock 138826682553952 released on /root/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-mpnet-base-cos-v1/tokenizer_config.json.lock
DEBUG:filelock:Attempting to acquire lock 138826682561056 on /root/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-mpnet-base-cos-v1/train_script.py.lock
DEBUG:filelock:Lock 138826682561056 acquired on /root/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-mpnet-base-cos-v1/train_script.py.lock


DEBUG:filelock:Attempting to release lock 138826682561056 on /root/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-mpnet-base-cos-v1/train_script.py.lock
DEBUG:filelock:Lock 138826682561056 released on /root/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-mpnet-base-cos-v1/train_script.py.lock
DEBUG:filelock:Attempting to acquire lock 138826682562832 on /root/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-mpnet-base-cos-v1/vocab.txt.lock
DEBUG:filelock:Lock 138826682562832 acquired on /root/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-mpnet-base-cos-v1/vocab.txt.lock


DEBUG:filelock:Attempting to release lock 138826682562832 on /root/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-mpnet-base-cos-v1/vocab.txt.lock
DEBUG:filelock:Lock 138826682562832 released on /root/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-mpnet-base-cos-v1/vocab.txt.lock
DEBUG:filelock:Attempting to acquire lock 138826682562544 on /root/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-mpnet-base-cos-v1/modules.json.lock
DEBUG:filelock:Lock 138826682562544 acquired on /root/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-mpnet-base-cos-v1/modules.json.lock


DEBUG:filelock:Attempting to release lock 138826682562544 on /root/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-mpnet-base-cos-v1/modules.json.lock
DEBUG:filelock:Lock 138826682562544 released on /root/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-mpnet-base-cos-v1/modules.json.lock
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda


In [7]:
def get_embeddings(texts):
  embed_texts = []

  for item in texts:
    embed_item = model.encode(item)
    vector = embed_item.tolist()
    embed_texts.append(vector)

  return embed_texts

In [8]:
def get_embedding(text):
  embed_text = model.encode(text)
  vector_text = embed_text.tolist()

  return vector_text

In [9]:
def my_hash(s):
    # Return the MD5 hash of the input string as a hexadecimal string
    return hashlib.md5(s.encode()).hexdigest()

In [10]:
def prepare_for_pinecone(texts):
    # Get the current UTC date and time
    now = datetime.utcnow()

    # Generate vector embeddings for each string in the input list
    embeddings = get_embeddings(texts)

    # Create tuples of (hash, embedding, metadata) for each input string and its corresponding vector embedding
    # The my_hash() function is used to generate a unique hash for each string, and the datetime.utcnow() function is used to generate the current UTC date and time
    return [
        (
            my_hash(text),  # A unique ID for each string, generated using the my_hash() function
            embedding,  # The vector embedding of the string
            dict(text=text, date_uploaded=now)  # A dictionary of metadata, including the original text and the current UTC date and time
        )
        for text, embedding in zip(texts, embeddings)  # Iterate over each input string and its corresponding vector embedding
    ]


In [11]:
def upload_texts_to_pinecone(texts, batch_size=None, show_progress_bar=False):
    # Call the prepare_for_pinecone function to prepare the input texts for indexing
    total_upserted = 0
    if not batch_size:
        batch_size = len(texts)

    _range = range(0, len(texts), batch_size)
    for i in tqdm(_range) if show_progress_bar else _range:
        batch = texts[i: i + batch_size]
        prepared_texts = prepare_for_pinecone(batch)

        # Use the upsert() method of the index object to upload the prepared texts to Pinecone
        total_upserted += index.upsert(
            prepared_texts,
        )['upserted_count']

    return total_upserted

In [12]:
def query_from_pinecone(query, top_k=3):
    # get embedding from THE SAME embedder as the documents
    query_embedding = get_embedding(query)

    return index.query(
      vector=query_embedding,
      top_k=top_k,
      include_metadata=True   # gets the metadata (dates, text, etc)
    ).get('matches')

In [ ]:
print(get_embedding('What is a pod?'))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[0.07525831460952759, -0.07845176756381989, -0.023035351186990738, -0.045587241649627686, -0.007826492190361023, -0.005947474390268326, 0.03149855136871338, -0.009010685607790947, -0.03187817335128784, 0.05668720602989197, -0.0015097910072654486, 0.03774908557534218, -0.015482758171856403, 0.06225806474685669, 0.029698090627789497, -0.005990678910166025, 0.03953726962208748, 0.039502933621406555, -0.053056392818689346, -0.010370018891990185, 0.05207604542374611, -0.04295722022652626, 0.0023577981628477573, 0.025966297835111618, -0.014239429496228695, -0.027380429208278656, 0.03552810102701187, 0.008274461142718792, -0.033636607229709625, 0.01154892798513174, 0.04147268459200859, -0.05350305140018463, -0.04745292291045189, -0.013604004867374897, 0.0071015446446835995, 0.004327153321355581, 0.024946987628936768, -0.028526032343506813, -0.04786953330039978, -0.004866709001362324, 0.04800791293382645, 0.020588574931025505, 0.001880371361039579, -0.025820529088377953, 0.015968501567840576, 

## Upload PDF to Pinecone vectorDB

In [ ]:
import PyPDF2

with open('/content/data/cl-oreilly-kubernetes-patterns-ebook-399085-202306-en.pdf', 'rb') as file:

    reader = PyPDF2.PdfReader(file)

    pdf = ''

    for page_num in range(15, len(reader.pages)):
        page = reader.pages[page_num]
        text = page.extract_text()
        pdf += '\n\n' + text
        #pdf += '\n\n' + text[text.find(' ]')+2:]

662209


In [ ]:
print(pdf[:100])



Preface
With the mainstream adoption of microservices and containers in recent years, the
way we d


In [ ]:
# Only keep documents of at least 100 characters split by a custom delimiter
pages = pdf.split('\n\n')
paragraphs = []
for page in pages:
  paragraphs.append(page.split('.\n'))

flattened_list = [item + '.' for sublist in paragraphs for item in sublist]

split = list(filter(lambda x: len(x) > 10, flattened_list))

print(f'custom delimiter approach has {len(split)} documents')

custom delimiter approach has 2229 documents


In [ ]:
print(split[102])

Pods
Looking  at the characteristics of containers, we can see that they are a perfect match
for implementing the microservices principles. A container image provides a single
unit of functionality, belongs to a single team, has an independent release cycle,
and provides deployment and runtime isolation. Most of the time, one microservice
corresponds to one container image.


In [ ]:
upload_texts_to_pinecone(split, batch_size=32, show_progress_bar=True)

  0%|          | 0/70 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

## CrossEncoder

In [13]:
"""
This example computes the score between a query and all possible
sentences in a corpus using a Cross-Encoder for semantic textual similarity (STS).
It output then the most similar sentences for the given query.
"""
from sentence_transformers.cross_encoder import CrossEncoder
import numpy as np
from torch import nn

# Pre-trained cross encoder
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2')

DEBUG:filelock:Attempting to acquire lock 138830392817840 on /root/.cache/huggingface/hub/models--cross-encoder--ms-marco-MiniLM-L-12-v2/blobs/bbc41782cb3876f78241487991bd9c17fb7aa1b4.lock
DEBUG:filelock:Lock 138830392817840 acquired on /root/.cache/huggingface/hub/models--cross-encoder--ms-marco-MiniLM-L-12-v2/blobs/bbc41782cb3876f78241487991bd9c17fb7aa1b4.lock


DEBUG:filelock:Attempting to release lock 138830392817840 on /root/.cache/huggingface/hub/models--cross-encoder--ms-marco-MiniLM-L-12-v2/blobs/bbc41782cb3876f78241487991bd9c17fb7aa1b4.lock
DEBUG:filelock:Lock 138830392817840 released on /root/.cache/huggingface/hub/models--cross-encoder--ms-marco-MiniLM-L-12-v2/blobs/bbc41782cb3876f78241487991bd9c17fb7aa1b4.lock
DEBUG:filelock:Attempting to acquire lock 138826573434208 on /root/.cache/huggingface/hub/models--cross-encoder--ms-marco-MiniLM-L-12-v2/blobs/207bb14d184b7728b7c2a68c685678aab636ae301a46f99fe05e5ffdae89e4d8.lock
DEBUG:filelock:Lock 138826573434208 acquired on /root/.cache/huggingface/hub/models--cross-encoder--ms-marco-MiniLM-L-12-v2/blobs/207bb14d184b7728b7c2a68c685678aab636ae301a46f99fe05e5ffdae89e4d8.lock


DEBUG:filelock:Attempting to release lock 138826573434208 on /root/.cache/huggingface/hub/models--cross-encoder--ms-marco-MiniLM-L-12-v2/blobs/207bb14d184b7728b7c2a68c685678aab636ae301a46f99fe05e5ffdae89e4d8.lock
DEBUG:filelock:Lock 138826573434208 released on /root/.cache/huggingface/hub/models--cross-encoder--ms-marco-MiniLM-L-12-v2/blobs/207bb14d184b7728b7c2a68c685678aab636ae301a46f99fe05e5ffdae89e4d8.lock
DEBUG:filelock:Attempting to acquire lock 138826670080928 on /root/.cache/huggingface/hub/models--cross-encoder--ms-marco-MiniLM-L-12-v2/blobs/2fd98132fd4620f90908272d9a9e6b2626e83491.lock
DEBUG:filelock:Lock 138826670080928 acquired on /root/.cache/huggingface/hub/models--cross-encoder--ms-marco-MiniLM-L-12-v2/blobs/2fd98132fd4620f90908272d9a9e6b2626e83491.lock


DEBUG:filelock:Attempting to release lock 138826670080928 on /root/.cache/huggingface/hub/models--cross-encoder--ms-marco-MiniLM-L-12-v2/blobs/2fd98132fd4620f90908272d9a9e6b2626e83491.lock
DEBUG:filelock:Lock 138826670080928 released on /root/.cache/huggingface/hub/models--cross-encoder--ms-marco-MiniLM-L-12-v2/blobs/2fd98132fd4620f90908272d9a9e6b2626e83491.lock
DEBUG:filelock:Attempting to acquire lock 138826574153232 on /root/.cache/huggingface/hub/models--cross-encoder--ms-marco-MiniLM-L-12-v2/blobs/fb140275c155a9c7c5a3b3e0e77a9e839594a938.lock
DEBUG:filelock:Lock 138826574153232 acquired on /root/.cache/huggingface/hub/models--cross-encoder--ms-marco-MiniLM-L-12-v2/blobs/fb140275c155a9c7c5a3b3e0e77a9e839594a938.lock


DEBUG:filelock:Attempting to release lock 138826574153232 on /root/.cache/huggingface/hub/models--cross-encoder--ms-marco-MiniLM-L-12-v2/blobs/fb140275c155a9c7c5a3b3e0e77a9e839594a938.lock
DEBUG:filelock:Lock 138826574153232 released on /root/.cache/huggingface/hub/models--cross-encoder--ms-marco-MiniLM-L-12-v2/blobs/fb140275c155a9c7c5a3b3e0e77a9e839594a938.lock
DEBUG:filelock:Attempting to acquire lock 138826682562736 on /root/.cache/huggingface/hub/models--cross-encoder--ms-marco-MiniLM-L-12-v2/blobs/e7b0375001f109a6b8873d756ad4f7bbb15fbaa5.lock
DEBUG:filelock:Lock 138826682562736 acquired on /root/.cache/huggingface/hub/models--cross-encoder--ms-marco-MiniLM-L-12-v2/blobs/e7b0375001f109a6b8873d756ad4f7bbb15fbaa5.lock


DEBUG:filelock:Attempting to release lock 138826682562736 on /root/.cache/huggingface/hub/models--cross-encoder--ms-marco-MiniLM-L-12-v2/blobs/e7b0375001f109a6b8873d756ad4f7bbb15fbaa5.lock
DEBUG:filelock:Lock 138826682562736 released on /root/.cache/huggingface/hub/models--cross-encoder--ms-marco-MiniLM-L-12-v2/blobs/e7b0375001f109a6b8873d756ad4f7bbb15fbaa5.lock
INFO:sentence_transformers.cross_encoder.CrossEncoder:Use pytorch device: cuda


In [14]:
def get_results_from_pinecone(query, top_k=3, re_rank=False, verbose=True):

    results_from_pinecone = query_from_pinecone(query, top_k=top_k)
    if not results_from_pinecone:
        return []

    if verbose:
        print("Query:", query)


    final_results = []

    if re_rank:
        if verbose:
            print('Document ID (Hash)\t\tRetrieval Score\tCE Score\tText')

        sentence_combinations = [[query, result_from_pinecone['metadata']['text']] for result_from_pinecone in results_from_pinecone]

        # Compute the similarity scores for these combinations
        similarity_scores = cross_encoder.predict(sentence_combinations, activation_fct=nn.Sigmoid())

        # Sort the scores in decreasing order
        sim_scores_argsort = reversed(np.argsort(similarity_scores))

        # Print the scores
        for idx in sim_scores_argsort:
            result_from_pinecone = results_from_pinecone[idx]
            final_results.append(result_from_pinecone)
            if verbose:
                print(f"{result_from_pinecone['id']}\t{result_from_pinecone['score']:.2f}\t{similarity_scores[idx]:.2f}\t{result_from_pinecone['metadata']['text'][:50]}")
        return final_results

    if verbose:
        print('Document ID (Hash)\t\tRetrieval Score\tText')
    for result_from_pinecone in results_from_pinecone:
        final_results.append(result_from_pinecone)
        if verbose:
            print(f"{result_from_pinecone['id']}\t{result_from_pinecone['score']:.2f}\t{result_from_pinecone['metadata']['text'][:50]}")

    return final_results

In [15]:
query = "What is a pod?"

final_results = get_results_from_pinecone(query, top_k=3, re_rank=True, verbose=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Query: What is a pod?
Document ID (Hash)		Retrieval Score	CE Score	Text


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

9933e131eaa38b9e0e0e3b7105b9ba6a	0.72	1.00	A Pod is the atom of Kubernetes where your applica
0b19e9e1467ffcb58b232f1c7c254363	0.71	1.00	However, most cloud native platforms offer another
6749189b7b19cf2c0caf1f25fa4dc70e	0.71	1.00	The Pod is such a fundamental primitive that it is


In [16]:
print(final_results)

[{'id': '9933e131eaa38b9e0e0e3b7105b9ba6a',
 'metadata': {'date_uploaded': datetime.datetime(2023, 11, 13, 10, 24, 37, 505440),
              'text': 'A Pod is the atom of Kubernetes where your application '
                      'lives, but you don’t access\n'
                      'Pods directly—that is where Services enter the scene.'},
 'score': 0.720226705,
 'values': []}, {'id': '0b19e9e1467ffcb58b232f1c7c254363',
 'metadata': {'date_uploaded': datetime.datetime(2023, 11, 13, 10, 24, 37, 505440),
              'text': 'However, most cloud native platforms offer another '
                      'primitive for managing the life‐\n'
                      'cycle of a group of containers—in Kubernetes, it is '
                      'called a Pod. A Pod is an atomic\n'
                      'unit of scheduling, deployment, and runtime isolation '
                      'for a group of containers. All\n'
                      'containers in a Pod are always scheduled to the same '
       